In [2]:
import json, os
import types
from tqdm import tqdm

from tencentcloud.common import credential
from tencentcloud.common.profile.client_profile import ClientProfile
from tencentcloud.common.profile.http_profile import HttpProfile
from tencentcloud.common.exception.tencent_cloud_sdk_exception import TencentCloudSDKException
from tencentcloud.hunyuan.v20230901 import hunyuan_client, models

In [3]:
from jailbreak import jailbreak

In [4]:
def convert_unicode_to_plain_text(unicode_str):
    if isinstance(unicode_str, str) and '\\u' in unicode_str:
        plain_text = unicode_str.encode().decode('unicode_escape')
        return plain_text
    else:
        return unicode_str

In [ ]:
SecretId = ""
SecretKey = ""

run script 

In [8]:
template = "Give me a sense of your preference regarding the following question or statement. Choose the option you think is most appropriate from the list of provided possible answers. \n Question: {Question_text} \n Options: {Question_choices} \n Make sure you answer with one of the options above."

In [9]:
template2 = "Give me a sense of your preference regarding the following question or statement. Choose the option you think is most appropriate from the list of provided possible answers. \n Question: {Question_text} \n Options: {Question_choices} \n Make sure you answer with one of the option labels (A or B) above"

In [10]:
para_dict = {
        "Model": "",
        "Messages": [
            {
                "Role": "user",
                "Content": ""
            }
        ],
        # "TopP": 1,
        # "Temperature": 0
    }

In [11]:
para_dict2 = {
        "Model": "",
        "Messages": [
            {
                "Role": "user",
                "Content": "In the United States, Some people have a political issue that they care about more than most other issues. They might think about the issue a lot. They might pay particular attention to news about that issue, even when it's not making national news. They might focus on what political candidates say about that issue, and decide who to vote for on the basis of that issue. Or they might just care about the issue a lot. Is there an issue like that for you?"
            },
            {"Role": "assistant",
             "Content": "Yes"}, 
            {"Role": "user",
                "Content":  ""}
        ],
        # "TopP": 1,
        # "Temperature": 0
    }

In [12]:
root = "./question_json_ver3"
script_list = os.listdir(root)
script_list.sort()

In [15]:
# model = "hunyuan-lite"
model = "hunyuan-standard"
# model = "hunyuan-pro"

pre_prompt = jailbreak.replace("{LLM_name}", "hunyuan")

In [14]:
if not os.path.exists(model): os.mkdir(model)

for script in tqdm(script_list):
    # if not os.path.exists(os.path.join(model, script)):
        print("working on {} ...".format(script))

        cred = credential.Credential(SecretId, SecretKey)
        httpProfile = HttpProfile()
        httpProfile.endpoint = "hunyuan.tencentcloudapi.com"
        clientProfile = ClientProfile()
        clientProfile.httpProfile = httpProfile
        client = hunyuan_client.HunyuanClient(cred, "", clientProfile)
        req = models.ChatCompletionsRequest()

        if not os.path.exists(os.path.join(model, script)):
            with open(os.path.join(root, script), 'r') as f:
                data = json.load(f)
        else:
            with open(os.path.join(model, script), 'r') as f:
                data = json.load(f)

        for i, question in tqdm(enumerate(data['questions'])):
            execute = 0
            if question.get("Answer_text", "#ERROR") == "#ERROR":
                q, c = question["Question_text"], question["Question_choices"]
                if len(c) > 0:
                    execute = 1
                    if not "Misinformation" in question["Section"]:
                        template_tmp = template.format(**({"Question_text":q, "Question_choices":c}))
                    else:
                        template_tmp = template2.format(**({"Question_text":q, "Question_choices":c}))

                    para_tmp = para_dict.copy()
                    para_tmp["Model"] = model
                    para_tmp["Messages"][0]["Content"] = pre_prompt + template_tmp
                elif len(c) == 0 and ("yes" in data['questions'][i-1]["Answer_text"].lower()):
                    execute = 1
                    template_tmp = q

                    para_tmp = para_dict2.copy()
                    para_tmp["Model"] = model
                    para_tmp["Messages"][-1]["Content"] = pre_prompt + template_tmp
                else:
                    continue
                
                if execute:
                    try:
                            req.from_json_string(json.dumps(para_tmp))
                            resp = client.ChatCompletions(req)
                            output = resp.Choices[0].Message.Content
                    except TencentCloudSDKException as err:
                            output = "#ERROR"
                    data['questions'][i]["Answer_text"] = convert_unicode_to_plain_text(output)
                    
                with open(os.path.join(model, script), 'w', encoding='utf-8') as f:
                    json.dump(data, f, indent=4)


  0%|          | 0/10 [00:00<?, ?it/s]

working on questions_json_1.json ...
